In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool

In [2]:
from prep_func import prep, get_res_from_df, read_data, get_extra_features

In [3]:
from config import train_datapath, test_datapath, not_fit_cols, my_params, drops, model_path, solution_path

In [4]:
df_train, df_test = read_data(train_datapath, test_datapath, not_fit_cols)

In [5]:
df_train = get_extra_features(df_train)
df_test = get_extra_features(df_test)

In [6]:
%%time
full_train_poly = prep(df_train[['id', 'crop', '.geo']])
full_test_poly = prep(df_test[['id', '.geo']])

CPU times: user 50.7 s, sys: 338 ms, total: 51 s
Wall time: 52 s


In [7]:
res_train = get_res_from_df(full_train_poly, full_train_poly)
res_test = get_res_from_df(full_test_poly, full_train_poly)

  0%|          | 0/4830 [00:00<?, ?it/s]

  0%|          | 0/2071 [00:00<?, ?it/s]

In [8]:
t = res_train.merge(df_train.merge(
    full_train_poly[['id', 'lat', 'lon', 'kms', 'diff']], on='id', how='left'), how='right', on ='id')
t_test = res_test.merge(df_test.merge(
    full_test_poly[['id', 'lat', 'lon', 'kms', 'diff']], on='id', how='left'), how='right', on ='id')

In [9]:
sols = {}
for i in range(7):
    sols[i] = []

In [10]:
for i, params in enumerate(my_params):
    cb = CatBoostClassifier(**params)
    all_pool = Pool(t.drop(list(drops)+list(['crop', 'id', '.geo']), axis=1), t['crop'])
    cb.fit(all_pool,  verbose=500)
    cb.save_model(f'{model_path}/model2_{i}')
    for i in cb.classes_:
        sols[i].append(cb.predict_proba(t_test[cb.feature_names_])[:, i])

0:	learn: 1.7322274	total: 84.9ms	remaining: 1m 49s
500:	learn: 0.0279084	total: 26s	remaining: 40.7s
1000:	learn: 0.0119420	total: 52.4s	remaining: 14.9s
1285:	learn: 0.0087966	total: 1m 7s	remaining: 0us
0:	learn: 1.1646152	total: 90.9ms	remaining: 1m 1s
500:	learn: 0.0014248	total: 34.5s	remaining: 12.2s
677:	learn: 0.0009603	total: 47s	remaining: 0us


In [ ]:
for i in cb.classes_:
    t_test[i] = np.max(sols[i], axis=0)
t_test['crop'] = t_test[cb.classes_].idxmax(axis=1)
t_test[['id', 'crop']].to_csv(solution_path, index=False)